# Tutorial01 - Getting Started

## Overview

🚀 In this tutorial, we'll initialize XRFeitoria and add basic shapes.

## Import XRFeitoria

After [Installing][1], import ``XRfeitoria`` by

[1]: http://xrfeitoria.readthedocs.io/en/latest/installation.html

In [ ]:
import xrfeitoria as xf

## Choose engine

``XRFeitoria`` supports both ``Blender`` and ``Unreal Engine``. Choose your engine and replace the following ``engine_exec_path`` with your own engine path.

In [ ]:
engine_exec_path = 'C:/Program Files/Blender Foundation/Blender 3.3/blender.exe'

Then, initialize XRFeitoria by ``xf.init_blender`` or by ``xf.init_unreal``.

In [ ]:
from pathlib import Path

exec_path_stem = Path(engine_exec_path).stem.lower()
if 'blender' in exec_path_stem:
    xf_runner = xf.init_blender(exec_path=engine_exec_path, background=False, new_process=True)
elif 'unreal' in exec_path_stem:
    xf_runner = xf.init_unreal(exec_path=engine_exec_path, background=False, new_process=True)

✨ Now you can see a new Blender/Unreal Engine process has started.

The codes above returns a handler named ``xf_runner``. By using ``xf_runner``, you can:
- add basic shapes
- add cameras
- import actors from files
- render images

In this tutorial, we will demonstrate how to add basic shapes in the engine.

## Add shapes

The following codes add some basic shapes into the engine, such as cubes, spheres, cylinders, and cones. 

The functions in ``xf_runner.Shape`` enable you to directly use basic shapes and get rid of preparing assets by yourself.  

In [ ]:
_scale = 0.5
# add cubes
for i in range(3):
    xf_runner.Shape.spawn(name=f'cube_{i}', type='cube', location=(0, i, 0), scale=(_scale, _scale, _scale))

# add spheres
for i in range(3):
    xf_runner.Shape.spawn(name=f'sphere_{i}', type='sphere', location=(1, i, 0), scale=(_scale, _scale, _scale))

# add cylinders
for i in range(3):
    xf_runner.Shape.spawn(name=f'cylinder_{i}', type='cylinder', location=(2, i, 0), scale=(_scale, _scale, _scale))

✨ Running the codes above should give a result as depicted below.

![](http://file.bj.zoe.sensetime.com/resources/meihaiyi/xrfeitoria/pics/tutorial-01.png)

## Final step

🥳 This is a good start! XRFeitoria was initialized and multiple shapes were added in the engine. 

**Do remember** to will close the engine. 

In [ ]:
xf_runner.close()

Ref to [doc](http://file.bj.zoe.sensetime.com/resources/meihaiyi/xrfeitoria/docs/en/_build/html/init.html), you can always use ``with`` statement to ensure the engine is closed when the codes are finished.